In [1]:
import math
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import math
import matplotlib.pyplot as plt

In [2]:
lem = WordNetLemmatizer() #create lemmatizer

In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shirinharandi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
dictionary = pd.read_csv('../data/processedDict.csv')
dictionary['word'] = dictionary['word'].apply(lambda x: lem.lemmatize(x, pos='n'))

In [5]:
dictionary

,word,cat_lev1,cat_lev2,cat_lev3,ndf
0,information,business,professional_conduct_host,advice,0.008904
1,recommendation,business,professional_conduct_host,advice,0.008375
2,tip,business,professional_conduct_host,advice,0.008399
3,advice,business,professional_conduct_host,advice,0.007568
4,suggestion,business,professional_conduct_host,advice,0.007580
...,...,...,...,...,...
347,talking,social,social_interaction,talking,0.001961
348,chatting,social,social_interaction,talking,0.001408
349,moment,social,social_interaction,talking,0.001155
350,chat,social,social_interaction,talking,0.000553


In [6]:
filepath = '/Users/shirinharandi/Desktop/COMP0031/Data/en_reviews/Manchester.csv'
reviews = pd.read_csv(filepath)

# reviews = reviews['date']
reviews

table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

#strip punct of each review -> lemmatise -> output is list of words so join into sentences
reviews['comments'] = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))
reviews

reviews['date'] = pd.to_datetime(reviews['date'])

In [7]:
reviews

,Unnamed: 0,listing_id,id,date,reviewer_id,reviewer_name,comments,langs
0,0,68951,993691,2012-03-12,1602133,Leopoldo,comfy and cosy space with a welcome survival k...,en
1,1,68951,1305184,2012-05-19,2093255,Deepthi,we stayed with paul for 2 night on a business ...,en
2,2,68951,1369794,2012-05-29,337778,Graham,what a fantastic place we stayed for a week an...,en
3,3,68951,1470176,2012-06-12,1829716,Kira,the house is very comfortable and the host are...,en
4,4,68951,1607783,2012-07-02,323672,Peter,my friend and i had a great stay at paul and d...,en
...,...,...,...,...,...,...,...,...
132579,144022,42135683,604472776,2020-02-11,326617985,Effanga,steves place is quite basic but had everything...,en
132580,144023,42135683,605332428,2020-02-14,30514766,Gökçe,it wa great experience it is quite and big roo...,en
132581,144024,42135683,605817713,2020-02-15,103013271,Rich,wa beyond my expectation and super friendly,en
132582,144026,42167098,606283081,2020-02-16,221655974,Karl,well thought out character property extremely ...,en


In [8]:
# DELETE LATER
# mask = (reviews['date'] >= '2013-01-01') & (reviews['date'] < '2018-01-01')
# reviews = reviews.loc[mask].copy()
# reviews[:250000]

In [26]:
def get_trends_nice():
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2014-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2017-01-01', '2018-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()


    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    for word in start_year_words:
        f = get_freq(word, old_reviews_list)
        old_total += f
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": 0.0}, ignore_index=True)
        
    new_total = 0.0
    for word in end_year_words:
        f = get_freq(word, new_reviews_list)
        new_total += f
        if word in start_year_words:
            ind = out.index[out['word']==word]
#             example_ind = ind
#             print("die")
#             out['new_frequency'][ind] = f
            out.loc[ind,"new_frequency"] = f
        else:
            out = out.append({'word': word, 'old_frequency': 0.0, 'new_frequency': f}, ignore_index=True)
    
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [27]:
soci = get_trends_nice()
soci

,word,old_frequency,new_frequency,old_sum_count,new_sum_count
0,country,3.871201,75.788609,68382.493256,2.490500e+06
1,français,0.693147,0.000000,68382.493256,2.490500e+06
2,personality,2.079442,6.931472,68382.493256,2.490500e+06
3,convenient,22.586175,587.226996,68382.493256,2.490500e+06
4,curtain,0.693147,20.506733,68382.493256,2.490500e+06
...,...,...,...,...,...
13132,dragan,0.000000,1.791759,68382.493256,2.490500e+06
13133,unfortunate,0.000000,79.136562,68382.493256,2.490500e+06
13134,brass,0.000000,2.484907,68382.493256,2.490500e+06
13135,touristic,0.000000,2.772589,68382.493256,2.490500e+06


In [28]:
# soci = soci.sort_values(by=['frequency'])
soci[soci['word'] == "country"]

,word,old_frequency,new_frequency,old_sum_count,new_sum_count
0,country,3.871201,75.788609,68382.493256,2.490500e+06


In [29]:
# dictionaryWords = dictionary[(dictionary['word'] == "communication")]
dictionary_words = dictionary['word'].tolist()

In [32]:
test = soci[soci['word'].isin(dictionary_words)]

test['Nfrequency'] = test['new_frequency'] / test['new_sum_count']
test['Ofrequency'] = test['old_frequency'] / test['old_sum_count']
test['gain'] = np.where(test['Ofrequency'] test['Nfrequency'] / test['Ofrequency'])
test

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,Nfrequency,Ofrequency,giain
3,convenient,22.586175,587.226996,68382.493256,2.490500e+06,0.000236,0.000330,0.713874
10,sheet,2.079442,40.608002,68382.493256,2.490500e+06,0.000016,0.000030,0.536196
13,shared,4.564348,82.144716,68382.493256,2.490500e+06,0.000033,0.000067,0.494150
15,situated,6.238325,149.144427,68382.493256,2.490500e+06,0.000060,0.000091,0.656444
23,answer,4.852030,182.820957,68382.493256,2.490500e+06,0.000073,0.000071,1.034572
...,...,...,...,...,...,...,...,...
10149,roof,0.000000,28.419034,68382.493256,2.490500e+06,0.000011,0.000000,inf
10657,generosity,0.000000,4.158883,68382.493256,2.490500e+06,0.000002,0.000000,inf
11115,responds,0.000000,18.021827,68382.493256,2.490500e+06,0.000007,0.000000,inf
11207,sized,0.000000,50.599744,68382.493256,2.490500e+06,0.000020,0.000000,inf


In [14]:
busi = get_trends_nice("business")

TypeError: get_trends_nice() takes 0 positional arguments but 1 was given

In [ ]:
busi = busi.sort_values(by=['frequency'])
busi